In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[151]

{'text': 'The error:\nError: googleapi: Error 403: Access denied., forbidden\n│\nand\n│ Error: Error creating Dataset: googleapi: Error 403: Request had insufficient authentication scopes.\nFor this solution make sure to run:\necho $GOOGLE_APPLICATION_CREDENTIALS\necho $?\nSolution:\nYou have to set again the GOOGLE_APPLICATION_CREDENTIALS as Alexey did in the environment set-up video in week1:\nexport GOOGLE_APPLICATION_CREDENTIALS="<path/to/your/service-account-authkeys>.json',
 'section': 'Module 1: Docker and Terraform',
 'question': 'Terraform - Error creating Dataset: googleapi: Error 403: Request had insufficient authentication scopes',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
query = "Can I still enroll in the course if it has already started?"

In [7]:
index.fit(documents)

In [8]:
boost = {
    'question': 3.0,
    'section': 0.5
}

results = index.search(
    query=query,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first an

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="<my-super-secret-key>")

In [11]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user",
        "content": query
    }]
)

In [13]:
response.choices[0].message.content

"It depends on the institution and the specific course. Many universities and online platforms allow late enrollment within a certain timeframe, especially if the course is in its early stages. However, this might come with some caveats, such as catching up on missed material or potentially missing out on early assignments and participation grades.\n\nHere are some steps you can take:\n\n1. **Check the Enrollment Policy:** Look at the course syllabus or institution's website for their policies on late enrollment.\n   \n2. **Contact the Instructor or Administration:** Reach out directly to the course instructor or the admissions office to discuss your situation and see if late enrollment is possible. They may be willing to accommodate you.\n\n3. **Consider Online Options:** If the course is available online, there may be more flexibility regarding start dates and enrollment.\n\n4. **Evaluate the Workload:** Assess whether you can manage to catch up with the class. This depends on the du

In [14]:
def search(query):
    boost = {
        'question': 3.0,
        'section': 0.5
    }
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [15]:
search('How do I install Kafka?')

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'According to https://github.com/dpkp/kafka-python/\n“DUE TO ISSUES WITH RELEASES, IT IS SUGGESTED TO USE https://github.com/wbarnha/kafka-python-ng FOR THE TIME BEING”\nUse pip install kafka-python-ng instead',
  'section': 'Project',
  'question': 'How to fix the error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'"?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'sect

In [16]:
from elasticsearch import Elasticsearch

In [17]:
es = Elasticsearch('http://localhost:9200')

In [18]:
es.info()

ObjectApiResponse({'name': 'df9046a4709e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GY5_-7U6Qa6Twx8H2FBkNQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [19]:
idx_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [20]:
idx_name = "course-questions"
es.indices.create(index=idx_name, body=idx_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [21]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=idx_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [23]:
query

'Can I still enroll in the course if it has already started?'

In [35]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    resp = es.search(index=idx_name, body=search_query)
    
    result_docs = []
    
    for hit in resp['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [39]:
elastic_search("What is dbt?")

[{'text': "Use the PostgreSQL COPY FROM feature that is compatible with csv files\nCOPY table_name [ ( column_name [, ...] ) ]\nFROM { 'filename' | PROGRAM 'command' | STDIN }\n[ [ WITH ] ( option [, ...] ) ]\n[ WHERE condition ]",
  'section': 'Module 4: analytics engineering with dbt',
  'question': 'What is the fastest way to upload taxi data to dbt-postgres?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Create a new branch to edit. More on this can be found here in the dbt docs.',
  'section': 'Module 4: analytics engineering with dbt',
  'question': 'Dbt+git - Main branch is “read-only”',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Answer: In short, it’s your repository on github, gitlab, bitbucket, etc\nIn long, your repository or any other location you have your code where a reasonable person would look at it and think yes, you went through the week and exercises.',
  'section': 'General course-related questions',
  'question': 'Homework - What is the homework 